In [ ]:
import numpy as np
import heapq
import pandas as pd
import matplotlib.pyplot as plt

8 QUEENS PROBLEM

STATE = numpy array of positions of queens on the chess board. \
if pos is the position of the queen then row and column are pos//dim and pos%dim \
where dim is the dimension of the board

In [ ]:
dim = 8
total = dim*dim

def get_neighbours(state : np.ndarray) -> list[np.ndarray]:
    neighbours = []
    for pos in range(total):
        r, c = pos//dim, pos%dim 
        conflict = False
        for queen in state:
            qr, qc = queen//dim, queen%dim
            if qr==r or qc==c or abs(qr-r)==abs(qc-c):
                conflict = True
                break
        
        if not conflict:
            new_state = np.append(state, pos)
            new_state.sort()
            neighbours.append(new_state)
    
    return neighbours

general A star for this problem. \
cost = 1 and g equal to number of edges traversed in the graph \
which is equal to number of queens in the present state

In [ ]:
def genA_star(initial : np.ndarray, heuristic_fn : callable) -> tuple[int, np.ndarray]:
    numNodesExpanded = 0
    open_list = []
    closed_list = set()
    heapq.heappush(open_list, (0 + heuristic_fn(initial), tuple(initial))) 
    g_scores = { tuple(initial) : 0 }

    while open_list:
        _, currStateTuple = heapq.heappop(open_list)
        if currStateTuple in closed_list:
            continue

        closed_list.add(currStateTuple)
        currState = np.array(currStateTuple)

        if currState.size == dim:
            return numNodesExpanded, currState.astype(int)

        numNodesExpanded+=1
        neighbours = get_neighbours(currState)

        for nextState in neighbours:
            nextStateTuple = tuple(nextState)
            if nextStateTuple in closed_list:
                continue

            if nextStateTuple not in g_scores:
                g_scores[nextStateTuple] = nextState.size
                heapq.heappush(open_list, (nextState.size + heuristic_fn(nextState), nextStateTuple))
         
    return 0, initial




Heuristics

In [ ]:
def heuristic_1(state : np.ndarray):
    return 0 

def heuristic_2(state : np.ndarray):
    return dim - state.size

def heuristic_3(state : np.ndarray):
    safe_count = 0 
    for pos in range(total):
        r, c = pos//dim, pos%dim
        safe = True
        for queen in state:
            qr, qc = queen//dim, queen%dim
            if qr==r or qc==c or abs(qr-r)==abs(qc-c):
                safe = False
                break
        
        if safe:
            safe_count+=1
    
    return safe_count

In [ ]:
def use_heuristic(heuristic_name, heuristic_fn : callable):
        initial = np.array([])
        numNodesExpanded, goal = genA_star(initial, heuristic_fn) 
        print(f"Number of nodes expanded using {heuristic_name} : {numNodesExpanded}")
        chessBoard = np.zeros((dim, dim), dtype = int)
        for queen in goal:
            qr, qc = queen//dim, queen%dim
            chessBoard[qr][qc] = 1
        for r in range(dim):
            row = ""
            for c in range(dim):
                if chessBoard[r][c] == 0:
                    row+="O "
                else:
                    row+="X "
            print(row)
        
        print("-----------------------------------")
        return

if __name__ == "__main__":
    use_heuristic("zero heuristic", heuristic_1) 
    use_heuristic("remaining queens heuristic", heuristic_2)
    use_heuristic("number of safe places left heuristic", heuristic_3)

Neural Network

In [ ]:
# Function to one-hot encode the target variable into the 10 classes (0-9)
# Input shape: (N,),    Output: (N, 10)
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, np.max(Y)+1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y

# Prepare data sets
def get_inputs(dim: int = 2):
    if dim == 1:
        return np.array([[0], [1]])
    array = get_inputs(dim-1)
    zeroes = np.zeros((2**(dim-1), 1))
    ones = np.ones((2**(dim-1), 1))
    arr1 = np.concatenate((zeroes, array), axis=1)
    arr2 = np.concatenate((ones, array), axis=1)

    return np.concatenate((arr1, arr2), axis=0)

In [ ]:
# Loading the MNIST dataset
train_data=pd.read_csv(r"./mnist_train.csv")
test_data=pd.read_csv(r"./mnist_test.csv")

# Preprocessing the data
train_data=train_data.to_numpy()    # train_data shape: (60000, 785)
test_data=test_data.to_numpy()      # test_data shape: (10000, 785)

X_train=train_data[:,1:]            # X_train shape: (60000, 784)
y_train=train_data[:,0]             # y_train shape: (60000,)
X_test=test_data[:,1:]              # X_test shape: (10000, 784)
y_test=test_data[:,0]               # y_test shape: (10000,)

X_train = X_train / 255.0           # Normalizing the data
X_test = X_test / 255.0

one_hot_y_train = one_hot(y_train)  # one_hot_y_train shape: (60000, 10)
one_hot_y_test = one_hot(y_test)    # one_hot_y_test shape: (10000, 10)



In [ ]:
class FNN:
    def __init__(self, layers_size, loss="ce", activation="relu", learning_rate=0.01) -> None:
        self.layers_size = layers_size
        self.loss = loss
        self.activation = activation
        self.learning_rate = learning_rate
        self.L = len(layers_size)

        self.weights = [np.random.randn(layers_size[i], layers_size[i+1])*0.01 for i in range(self.L-1)]
        self.biases = [np.zeros((1, layers_size[i+1])) for i in range(self.L-1)]

    def activation_fn(self, x, deriv=False):
        if self.activation == "sigmoid":
            sig = 1/(1+np.exp(-x))
            return sig if not deriv else sig*(1-sig)
        elif self.activation == "relu":
            return np.maximum(x, 0) if not deriv else (x>0).astype(float)
    
    def loss_fn(self, y_pred, y_true, deriv=False):
        if self.loss == "tss":
            loss = 0.5*np.sum((y_pred - y_true)**2)
            return loss if not deriv else y_pred-y_true
        elif self.loss == "ce":
            loss = -np.sum(y_true*np.log(y_pred + 1e-9))
            return loss if not deriv else y_pred-y_true
    
    
    def softmax(self, x):
        ret = np.exp(x - np.max(x, axis=1, keepdims=True))
        return  ret/(np.sum(ret, axis=1, keepdims=True))

    def forward(self, X):
        A = [X]
        Z = []

        for i in range(self.L-2):
            z = np.dot(A[-1], self.weights[i]) + self.biases[i]
            Z.append(z) 
            A.append(self.activation_fn(z))
        
        z = np.dot(A[-1], self.weights[-1]) + self.biases[-1]
        Z.append(z) 
        A.append(self.softmax(z))

        return A, Z
    
    def backward(self, X_train, y_train):
        m = X_train.shape[0]
        A, Z = self.forward(X_train) 
        y_pred = A[-1]

        dA = self.loss_fn(y_pred, y_train, True)
        
        dWs = []
        dbs = []
        
        for i in range(self.L-2, -1, -1):
            dZ = dA if (self.loss == "ce" and i==self.L-2) else dA*self.activation_fn(Z[i], True)
            dW = np.dot(A[i].T, dZ)/m
            db = np.sum(dZ, axis = 0, keepdims=True)/m
            dWs.append(dW)
            dbs.append(db)
            dA = np.dot(dZ, self.weights[i].T) 
        
        dWs.reverse()
        dbs.reverse()
        for i in range(self.L-1):
            self.weights[i]-=(self.learning_rate*dWs[i])
            self.biases[i]-=(self.learning_rate*dbs[i])
        
        return 
    
    def train(self, X, y, epochs=10, batch_size=32, print_in = 2):
        m = X.shape[0]
        losses = []
        for epoch in range(epochs):
            perm = np.random.permutation(m)
            X_perm, y_perm = X[perm], y[perm]
            for i in range(0, m, batch_size):
                X_batch = X_perm[i : i+batch_size]
                y_batch = y_perm[i : i+batch_size]
                self.backward(X_batch, y_batch)
            
            if epoch%print_in == 0:
                loss = self.loss_fn(self.forward(X)[0][-1], y)
                losses.append(loss)
                print(f"Epoch : {epoch} => Loss : {loss}")
        return losses
    
    def predict(self, X):
        return np.argmax(self.forward(X)[0][-1], axis=1)
    
    def draw_graph(self, y_axis, x_axis):
        plt.plot(x_axis, y_axis)
    

In [ ]:
model = FNN((784, 128, 64, 10), "ce", "relu", 0.01)
print_in = 2
epochs = 10
batch_size = 32

In [ ]:
losses = model.train(X_train, one_hot_y_train, epochs, batch_size, print_in)
x_axis = np.arange(0, epochs, print_in)
model.draw_graph(losses, x_axis)

y_predict = model.predict(X_test)
accuracy = np.mean(y_predict == y_test)
print(f"Accuracy => {accuracy*100}")

In [ ]:
# XOR data set 
input_XOR = get_inputs(2)
output_XOR = np.array([0, 1, 1, 0])
output_XOR_one_hot = one_hot(output_XOR)

In [ ]:
XOR_model = FNN((2, 4, 2), "ce", "relu", 0.01)
epochs = 10000
batch_size = 4
print_in = 1000

In [ ]:
losses = XOR_model.train(input_XOR, output_XOR_one_hot, epochs, batch_size, print_in)
x_axis = np.arange(0, epochs, print_in)
XOR_model.draw_graph(losses, x_axis)

y_predict = XOR_model.predict(input_XOR)
accuracy = np.mean(y_predict == output_XOR)
print(f"Accuracy => {accuracy*100}")